## The analysis for this thesis was run from commit 87a5f02d7f526c4fe92909790999da5f26166328:
https://github.com/statsbomb/open-data/commit/87a5f02d7f526c4fe92909790999da5f26166328

From this commit, the dataframes have the following number of entries:
- df_competition: 36 entries
- df_match: 844 entries
- df_lineup: 25172 entries
- df_event: 3062353 entries
- df_freeze: 267024 entries
- df_tactic: 35335 entries
- df_related: 5953632 entrie

This notebook takes the StatsBomb json files and turns them into parquet files. These are extremely fast to load so good for this prototyping kind of analysis.

In [ ]:
import mplsoccer.statsbomb as sbapi
import pandas as pd
import os
import glob
import numpy as np

# Change these paths/ parameters
You will need to change these paths/ parameters depending on where the StatsBomb open-data is located, how and where you want to save the resulting data, and if you only want the new files to be processed.

In [ ]:
# open data folder is one folder down in the directory. To change if run elsewhere
STATSBOMB_DATA = os.path.join('..', '..', '..', 'open-data','data')
# save files in folder in current directory. To change if want to save elsewhere
DATA_FOLDER = os.path.join('..', '..', 'data', 'statsbomb')

# Get the data file paths

In [ ]:
event_links = glob.glob(os.path.join(STATSBOMB_DATA, 'events', '**', '*.json'),recursive=True)
lineup_links = glob.glob(os.path.join(STATSBOMB_DATA, 'lineups', '**', '*.json'),recursive=True)
match_links = glob.glob(os.path.join(STATSBOMB_DATA, 'matches', '**', '*.json'),recursive=True)
competition_path = os.path.join(STATSBOMB_DATA, 'competitions.json')

# Make the directory structure

In [ ]:
# make the directory structure
for folder in ['event_raw', 'related_raw', 'freeze_raw', 'tactic_raw', 'lineup_raw']:
    path = os.path.join(DATA_FOLDER, folder)
    if not os.path.exists(path):
        os.mkdir(path)

# Read the competition data

In [ ]:
df_competition = sbapi.read_competition(competition_path, warn=False)
# note there is a slight loss of data quality with timestamps, but these aren't relevant for analysis
# pandas has nanoseconds, which aren't supported in parquet (supports milliseconds)
df_competition.to_parquet(os.path.join(DATA_FOLDER, 'competition.parquet'), allow_truncated_timestamps=True)
df_competition.info()

# Keep a copy of the match dataframe

In [ ]:
match_path = os.path.join(DATA_FOLDER, 'match.parquet')
if os.path.exists(match_path):
    df_match_copy = pd.read_parquet(match_path).copy()
    update_files = True
else:
    update_files = False

# Read the match data

In [ ]:
match_dfs = [sbapi.read_match(file, warn=False) for file in match_links]
df_match = pd.concat(match_dfs)
# again there is a slight loss of quality when saving timestamps, but only relevant for last_updated
df_match.to_parquet(os.path.join(DATA_FOLDER, 'match.parquet'), allow_truncated_timestamps=True)
df_match.info()

# Check which games have been updated

In [ ]:
if update_files == True:
    df_match_copy = df_match[['match_id', 'last_updated']].merge(df_match_copy[['match_id', 'last_updated']],
                                                                 how='left', suffixes=['', '_old'], on='match_id')
    df_match_copy = df_match_copy[df_match_copy.last_updated.dt.floor('ms') != df_match_copy.last_updated_old].copy()
    to_update = df_match_copy.match_id.unique()
    
    # get array of event links to update - based on whether they have been updated in the match json
    event_link_ids = np.array([int(os.path.splitext(os.path.basename(link))[0]) for link in event_links])
    event_to_update = [link in to_update for link in event_link_ids]
    event_links = np.array(event_links)[event_to_update]
    
    # get array of lineup links to update - based on whether they have been updated in the match jsons
    lineup_link_ids = np.array([int(os.path.splitext(os.path.basename(link))[0]) for link in lineup_links])
    lineup_to_update = [link in to_update for link in lineup_link_ids]
    lineup_links = np.array(lineup_links)[lineup_to_update]

# Read the lineup data

In [ ]:
LINEUP_FOLDER = os.path.join(DATA_FOLDER, 'lineup_raw')
# loop through all the changed links and store as parquet files - small and fast files
for file in lineup_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        print('Trying:', file)
        df_lineup = sbapi.read_lineup(file, warn=False)
        df_lineup.to_parquet(os.path.join(LINEUP_FOLDER, save_path))
    except:
        print('Skipping:', file)
        pass

Convert to a single dataframe

In [ ]:
# players with split ids
to_replace = {18103: 38522,  # Dietmar Hamann
              17275: 4656,  # Hannah Jayne Blundell
              17524: 4655,  # Jennifer Beattie
              10172: 4644,  # Jill Scott
              4634: 5088,  # Crystal Dunn
              4639: 10395}  # Maren Mjelde

In [ ]:
if len(lineup_links) == 0:
    print('No update')
else:
    lineup_files = glob.glob(os.path.join(LINEUP_FOLDER, '*.parquet'))
    df_lineup = pd.concat([pd.read_parquet(file) for file in lineup_files])
    # replace some ids that appear to be split
    df_lineup.player_id.replace(to_replace, inplace=True)
    df_lineup.to_parquet(os.path.join(DATA_FOLDER, 'lineup.parquet'))
    df_lineup.info()

# Read the event data

In [ ]:
# loop through all the changed links and store as parquet files - small and fast files
for file in event_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        print('Trying:', file)
        dict_event = sbapi.read_event(file, warn=False)
        # save to parquet files
        # using the dictionary key to access the dataframes from the dictionary
        dict_event['event'].to_parquet(os.path.join(DATA_FOLDER, 'event_raw', save_path))
        dict_event['related_event'].to_parquet(os.path.join(DATA_FOLDER, 'related_raw', save_path))
        dict_event['shot_freeze_frame'].to_parquet(os.path.join(DATA_FOLDER, 'freeze_raw', save_path))
        dict_event['tactics_lineup'].to_parquet(os.path.join(DATA_FOLDER, 'tactic_raw', save_path))
    except:
        print('Skipping:', file)
        pass

If updating the event dataframe get a list of ids to update

In [ ]:
event_files = glob.glob(os.path.join(DATA_FOLDER, 'event_raw', '*.parquet'))
if update_files:
    event_files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in event_files]
    ids_to_update = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]    

Function to load old dataframe and combine with updated parquet files

In [ ]:
def update(folder, file_type, ids_to_update):
    # get a list of parquet files to add to the old dataframe
    files = glob.glob(os.path.join(folder, f'{file_type}_raw', '*.parquet'))
    files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in files]
    mask_update = [match_id in ids_to_update for match_id in files_id]
    files = np.array(files)[mask_update]
    # load the old dataframe, filter out changed matches and add the new parquet files
    df_old = pd.read_parquet(os.path.join(folder, f'{file_type}.parquet'))
    df_old = df_old[~df_old.match_id.isin(ids_to_update)]
    df_new = pd.concat([pd.read_parquet(file) for file in files])
    df_old = pd.concat([df_old, df_new])
    return df_old

Single dataframe events

In [ ]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_event = update(DATA_FOLDER, 'event', ids_to_update)
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, null_counts=True)        
    else:
        df_event = pd.concat([pd.read_parquet(file) for file in event_files])
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, null_counts=True)

Single dataframe shot freeze frames

In [ ]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_freeze = update(DATA_FOLDER, 'freeze', ids_to_update)
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info(verbose=True, null_counts=True)      
    else:
        freeze_files = glob.glob(os.path.join(DATA_FOLDER, 'freeze_raw', '*.parquet'))
        df_freeze = pd.concat([pd.read_parquet(file) for file in freeze_files])
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info()

Single dataframe tactics

In [ ]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_tactic = update(DATA_FOLDER, 'tactic', ids_to_update)
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info(verbose=True, null_counts=True)      
    else:
        tactic_files = glob.glob(os.path.join(DATA_FOLDER, 'tactic_raw', '*.parquet'))
        df_tactic = pd.concat([pd.read_parquet(file) for file in tactic_files])
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info()

Single dataframe related events

In [ ]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_related = update(DATA_FOLDER, 'related', ids_to_update)
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, null_counts=True)     
    else:
        related_files = glob.glob(os.path.join(DATA_FOLDER, 'related_raw', '*.parquet'))
        df_related = pd.concat([pd.read_parquet(file) for file in related_files])
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, null_counts=True)